In [177]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf
from collections import Counter

%matplotlib inline

In [178]:
X_all = np.load('../hispanic_colrect/X.npy')
Y1 = np.load('../hispanic_colrect/Y1.npy')
Y2 = np.load('../hispanic_colrect/Y2.npy')
Y5 = np.load('../hispanic_colrect/Y5.npy')

In [179]:
# shuffle
np.random.seed(42)
idx = np.random.permutation(len(X_all))
X = X_all[idx]
Y = Y5[idx]

TEST_SET_SIZE = 6000

X_train, X_test = X[:-TEST_SET_SIZE], X[-TEST_SET_SIZE:]
Y_train, Y_test = Y[:-TEST_SET_SIZE].astype(int), Y[-TEST_SET_SIZE:].astype(int)

In [180]:
# Features Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
std_scaler = StandardScaler()
X_train[:,-4:] = std_scaler.fit_transform(X_train[:,-4:])
X_test[:,-4:] = std_scaler.transform(X_test[:,-4:])

/home/samuel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [181]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix

def results(classifier,threshold=0.5):
    Y_pred_test = classifier.predict(X_test)
    Y_pred_test = (Y_pred_test>=threshold).astype(int)
    #Y_pred_test = np.rint(Y_pred_test)
    print("Test accuracy score: " + str(accuracy_score(Y_test.astype(int), Y_pred_test)))
    print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
    matrix = confusion_matrix(Y_test.astype(int), Y_pred_test)
    tn, fp, fn, tp = matrix.ravel()
    ppv = tp/(tp+fp)
    npv = tn/(tn+fn)
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    g_mean = np.sqrt(sensitivity*specificity)
    print("PPV: " + str(ppv))
    print("NPV: " + str(npv))
    print("Sensitivity: " + str(sensitivity))
    print("Specificity: " + str(specificity))
    print("G-Mean: " + str(g_mean))
    print("Confusion matrix:\n" + str(matrix))

In [182]:
X_train[0,-4:]

array([-1.715922924097127, -0.2892952509005862, -0.46326593467355504,
       0.13496092138431548], dtype=object)

In [183]:
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.models import Sequential

model = Sequential()
#model.add(Dropout(0.25, input_shape=(X.shape[1],)))
model.add(Dense(400, input_dim=X.shape[1], activation=keras.layers.LeakyReLU(alpha=0.1)))
model.add(Dropout(0.25))

model.add(Dense(400, activation=keras.layers.LeakyReLU(alpha=0.1)))
model.add(Dropout(0.25))

model.add(Dense(400, activation=keras.layers.LeakyReLU(alpha=0.1)))
model.add(Dropout(0.25))


model.add(Dense(1, activation='tanh'))

In [184]:
#with tf.Session(graph=graph) as sess:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=5, batch_size=50)
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/5
40785/40785 [==============================] - 7s 182us/step - loss: 0.4401 - acc: 0.8078
Epoch 2/5
40785/40785 [==============================] - 6s 142us/step - loss: 0.4089 - acc: 0.8199
Epoch 3/5
40785/40785 [==============================] - 6s 143us/step - loss: 0.3989 - acc: 0.8232
Epoch 4/5
40785/40785 [==============================] - 6s 143us/step - loss: 0.3966 - acc: 0.8242
Epoch 5/5
6000/6000 [==============================] - 1s 184us/step

acc: 82.08%


In [186]:
results(model, threshold=0.7)

Test accuracy score: 0.815
ROC: 0.7590379594296044
PPV: 0.8909825033647375
NPV: 0.5953307392996109
Sensitivity: 0.8642297650130548
Specificity: 0.6538461538461539
G-Mean: 0.7517135810221545
Confusion matrix:
[[ 918  486]
 [ 624 3972]]


In [69]:
Y_pred_test = model.predict(X_test)

In [147]:
Y_pred_test[210:230]

array([[ 0.9780362 ],
       [ 0.9742021 ],
       [-0.05996058],
       [ 0.9298522 ],
       [ 0.97326124],
       [ 0.946064  ],
       [ 0.962197  ],
       [ 0.94709915],
       [ 0.9627532 ],
       [ 0.9947199 ],
       [ 0.7142358 ],
       [ 0.96079725],
       [ 0.9714728 ],
       [ 0.9769498 ],
       [ 0.8292747 ],
       [ 0.7820896 ],
       [ 0.48054105],
       [ 0.91638213],
       [ 0.9718184 ],
       [ 0.824868  ]], dtype=float32)

In [148]:
Y_test[210:230]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1])

In [12]:
#with graph.as_default():
Y_pred_test = model.predict(X_test)
Y_pred_test = np.rint(Y_pred_test)

In [13]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
#Y_pred_test = grid_search.predict(std_scaler.transform(X_test))
print(accuracy_score(Y_test.astype(int), Y_pred_test))
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred_test)))
confusion_matrix(Y_test.astype(int), Y_pred_test)

0.823833333333
ROC: 0.738294587901


array([[ 825,  612],
       [ 445, 4118]])

In [130]:
tn, fp, fn, tp = confusion_matrix(Y_test.astype(int), Y_pred_test).ravel()
ppv = tp/(tp+fp)
npv = tn/(tn+fn)
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print("PPV: " + str(ppv))
print("NPV: " + str(npv))
print("Sensitivity: " + str(sensitivity))
print("Specificity: " + str(specificity))

PPV: 0.873648505406
NPV: 0.655494933749
Sensitivity: 0.903133903134
Specificity: 0.58524704245
